In [1]:
from transformers import BertModel, BertTokenizer
import io

import numpy as np
from collections import namedtuple
import sys
from typing import List, Tuple, Dict, Set, Union
import torch
import torch.nn as nn
import torch.nn.utils
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from nltk import word_tokenize
import pickle
import timeit
from scipy import spatial

from evaluator import Evaluator
from vocab import Vocab, VocabEntry
from utils import read_corpus, pad_sents, batch_iter_bert

[nltk_data] Downloading package punkt to /Users/yuloucn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
words, defs, ft_dict = pickle.load( open( "../data/words_defs_dict_1M.train", "rb" ))

vocab = VocabEntry.from_corpus(defs, 1000000, 0)
for w in ft_dict:
    vocab.add(w)

number of word types: 23437, number of word types w/ frequency >= 0: 23437


In [3]:
def create_emb_layer(weights_matrix, src_pad_token_idx, non_trainable=True):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim, src_pad_token_idx)
    emb_layer.weight.data.copy_(torch.from_numpy(weights_matrix)) #figure out what is here
    if non_trainable:
        emb_layer.weight.requires_grad = False
    return emb_layer, num_embeddings, embedding_dim

class ModelEmbeddings(nn.Module): 
    """
    Class that converts input words to their embeddings.
    """

    def __init__(self, embed_size, vocab, fasttext_dict):
        """
        Init the Embedding layers.

        @param embed_size (int): Embedding size (dimensionality)
        @param vocab (VocabEntry)
        """
        super(ModelEmbeddings, self).__init__()

        self.embed_size = embed_size

        matrix_len = len(vocab)
        weights_matrix = np.zeros((matrix_len, self.embed_size))
        words_found = 0
        #print(len(vocab), weights_matrix.shape)
        for word, index in vocab.word2id.items():
            try:
                weights_matrix[index] = np.array(fasttext_dict[word])
                words_found += 1
            except KeyError:
                weights_matrix[index] = np.random.normal(scale=0.6, size=(self.embed_size,))

        # default values
        src_pad_token_idx = vocab['<pad>']
        self.source = create_emb_layer(weights_matrix, src_pad_token_idx, True)

In [4]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [5]:
class ReverseDictionary(nn.Module):

    def __init__(self, embed_dim, hidden_dim, vocab, ft_dict, freeze_bert = False):
        super(ReverseDictionary, self).__init__()
        #Instantiating BERT model object 
        
        self.ft_embedding = ModelEmbeddings(embed_dim, vocab, ft_dict)
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
#         Freeze bert layers
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
        
        #Classification layer
        self.lstm_fasttext = nn.LSTM(embed_dim, hidden_dim, batch_first = True)
        self.lin_layer = nn.Linear(hidden_dim+768, embed_dim)


    def forward(self, ft_input, lengths, bert_input, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        embedded = self.ft_embedding.source[0](ft_input).transpose(1,0)

        embedded = pack_padded_sequence(embedded, lengths, batch_first=True)
        output, (cn, hn) = self.lstm_fasttext(embedded)
        
        cont_reps, _ = self.bert_layer(bert_input, attention_mask = attn_masks)
        cls_rep = cont_reps[:, 0]

        toLinear = torch.cat([cls_rep, cn.squeeze(0)], 1)

        #Obtaining the representation of [CLS] head
        
        #feed cls_rep to -> fasttext layer
        projected = self.lin_layer(toLinear)

        return projected


In [17]:
model = ReverseDictionary(300, 300, vocab, ft_dict)
loss_function = nn.L1Loss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr = .001)

In [7]:
int_sents = vocab.words2indices(defs)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = max(len(x) for x in int_sents)
sents_ft_ids = int_sents
sents_bert_ids = []
masks = []
for d in defs:
    tokens = ['[CLS]'] + d + ['[SEP]']
    padded_tokens = tokens + ['[PAD]' for _ in range(max_len - len(tokens))]
    attn_mask = [1 if token != '[PAD]' else 0 for token in padded_tokens]
    seg_ids = [0 for _ in range(len(padded_tokens))]
    token_ids = tokenizer.convert_tokens_to_ids(padded_tokens)
    sents_bert_ids.append(token_ids)
    masks.append(attn_mask)
    

assert(len(sents_bert_ids) == len(defs))
assert(len(sents_bert_ids) == len(masks))
assert(len(masks) == len(words))
data = [(defs[i], len(defs[i]), sents_bert_ids[i], masks[i], words[i]) for i in range(len(defs))]

In [15]:
start = timeit.default_timer()
losses = []
BATCH_SIZE = 2

for epoch in range(5000):
    loss_cum = []
    for batch_defs, batch_lengths, batch_bert_ids, batch_masks, batch_words in \
                    batch_iter_bert(data, BATCH_SIZE, shuffle=False):
        model.zero_grad()
        #print(batch_lengths)
        batch_ft_ids = vocab.to_input_tensor(batch_defs, device = "cpu")
        batch_bert_ids = torch.tensor(batch_bert_ids, device = "cpu")
        batch_masks = torch.tensor(batch_masks, device = "cpu")
        tag_scores = model.forward(batch_ft_ids, batch_lengths, batch_bert_ids, batch_masks)
        
        y_pred = tag_scores[0].double()
        y_indices = torch.tensor([vocab[i] for i in batch_words])
        y_array = model.ft_embedding.source[0](y_indices).double()
        #print(y_array.shape)

        loss = loss_function(y_pred, y_array)
        print(loss)
        loss_cum.append(loss)
        loss.backward()
        optimizer.step() 
    lossavg = sum(loss_cum)/len(loss_cum)
    losses.append(loss)
    #print(epoch, lossavg, timeit.default_timer() - start)

tensor(0.2443, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.3441, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(2.4338, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(1.2702, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.6273, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(1.6615, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.7969, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.6948, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.5741, dtype=torch.float64, grad_fn=<L1LossBackward>)


KeyboardInterrupt: 

In [18]:
start = timeit.default_timer()
losses = []
BATCH_SIZE = 2


loss_cum = []
for batch_defs, batch_lengths, batch_bert_ids, batch_masks, batch_words in \
                batch_iter_bert(data, BATCH_SIZE, shuffle=False):
    for i in range(500):
        model.zero_grad()
        #print(batch_lengths)
        batch_ft_ids = vocab.to_input_tensor(batch_defs, device = "cpu")
        batch_bert_ids = torch.tensor(batch_bert_ids, device = "cpu")
        batch_masks = torch.tensor(batch_masks, device = "cpu")
        tag_scores = model.forward(batch_ft_ids, batch_lengths, batch_bert_ids, batch_masks)

        y_pred = tag_scores[0].double()
        y_indices = torch.tensor([vocab[i] for i in batch_words])
        y_array = model.ft_embedding.source[0](y_indices).double()
        #print(y_array.shape)

        loss = loss_function(y_pred, y_array)
        print(loss)
        loss_cum.append(loss)
        loss.backward()
        optimizer.step() 
    lossavg = sum(loss_cum)/len(loss_cum)
    losses.append(loss)
    #print(epoch, lossavg, timeit.default_timer() - start)

tensor(0.2355, dtype=torch.float64, grad_fn=<L1LossBackward>)


/Users/yuloucn/opt/miniconda3/envs/local_nmt/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/Users/yuloucn/opt/miniconda3/envs/local_nmt/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


tensor(0.2434, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.1535, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.1348, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0883, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.1155, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.2149, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.1107, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0871, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0705, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0659, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0638, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0551, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0524, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0500, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.1201, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0577, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0

tensor(0.0121, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0119, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0123, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0126, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0124, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0118, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0112, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0109, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0112, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0116, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0116, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0112, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0114, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0109, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0105, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0111, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0

tensor(0.0081, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0079, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0077, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0074, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0075, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0083, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0089, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0078, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0071, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0074, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0073, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0073, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0075, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0076, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0073, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0072, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0

tensor(0.0066, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0064, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0065, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0064, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0060, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0061, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0074, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0065, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0063, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0062, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0056, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0057, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0058, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0057, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0056, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0.0053, dtype=torch.float64, grad_fn=<L1LossBackward>)
tensor(0

In [19]:
eval = Evaluator()
model.zero_grad()

for epoch in range(5000):
    loss_cum = []
    for batch_defs, batch_lengths, batch_bert_ids, batch_masks, batch_words in \
                    batch_iter_bert(data, BATCH_SIZE, shuffle=False):
        model.zero_grad()
        #print(batch_lengths)
        batch_ft_ids = vocab.to_input_tensor(batch_defs, device = "cpu")
        batch_bert_ids = torch.tensor(batch_bert_ids, device = "cpu")
        batch_masks = torch.tensor(batch_masks, device = "cpu")
        tag_scores = model.forward(batch_ft_ids, batch_lengths, batch_bert_ids, batch_masks)
        
        y_pred = tag_scores[0].double()
        y_indices = torch.tensor([vocab[i] for i in batch_words])
        y_array = model.ft_embedding.source[0](y_indices).double()
        eval.top_ten_hundred(ft_dict, words[i], y_pred.detach().numpy())
    print(np.linalg.norm(ft_dict[words[i]]-y_pred.detach().numpy()))
#     print(np.linalg.norm(ft_dict[words[i]]-y_pred.detach().numpy()))
#     print(sorted(ft_dict.keys(), key=lambda word: spatial.distance.cosine(ft_dict[word], y_pred.detach().numpy())))
#     print(ft_dict['fault'].shape, y_pred.detach().numpy().shape)
#     print(loss)

NameError: name 'sents_bert_id' is not defined